<a href="https://colab.research.google.com/github/ChintPatel/CMPE258-HW5/blob/main/HW5_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Install
!pip install --upgrade pip
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl accelerate bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-orsokgbh/unsloth_a2666812515c46a0ab922d926aaa8fef
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-orsokgbh/unsloth_a2666812515c46a0ab922d926aaa8fef
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:


# 2️⃣ Imports
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset

# 3️⃣ Config & Load
MODEL_NAME    = "google/gemma-2-2b"
max_seq_length = 512
dtype          = torch.float16       # T4 favors fp16
load_in_4bit   = True                # QLoRA-style 4-bit weights

# This handles quantization under the hood
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name    = MODEL_NAME,
    max_seq_length= max_seq_length,
    dtype         = dtype,
    load_in_4bit  = load_in_4bit,
)

# 4️⃣ Patch in LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r                     = 8,
    target_modules       = ["q_proj", "v_proj"],
    lora_alpha           = 16,
    lora_dropout         = 0.05,
    bias                 = "none",
    use_gradient_checkpointing = "unsloth",
    random_state         = 42,
)


==((====))==  Unsloth 2025.4.1: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.1 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Load Alpaca-cleaned
ds_chat    = load_dataset("yahma/alpaca-cleaned")["train"]
train_chat = ds_chat.shuffle(42).select(range(1000))
eval_chat  = ds_chat.shuffle(42).select(range(100,300))

# Format into a single "text" field
EOS = tokenizer.eos_token
def format_chat(ex):
    instr = ex["instruction"]
    inp   = ex["input"]
    out   = ex["output"]
    prompt = (
        "### Instruction:\n" + instr +
        (("\n### Input:\n"+inp) if inp else "") +
        "\n### Response:\n" + out + EOS
    )
    return {"text": prompt}

# use the exact same format_chat you had, but map with batched=False
train_chat = train_chat.map(format_chat, batched=False, remove_columns=ds_chat.column_names)
eval_chat  = eval_chat.map (format_chat, batched=False, remove_columns=ds_chat.column_names)


# Trainer for chat
chat_args = TrainingArguments(
    per_device_train_batch_size   = 2,
    gradient_accumulation_steps   = 1,
    num_train_epochs              = 1,
    logging_steps                 = 50,
    save_steps                    = 200,
    fp16                          = True,
    output_dir                    = "./lora_chat"
)

trainer_chat = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = train_chat,
    eval_dataset        = eval_chat,
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    args                = chat_args,
)

trainer_chat.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 1,597,440/2,000,000,000 (0.08% trained)


Step,Training Loss
50,1.201200
100,1.210700
150,1.163200
200,1.186300
250,1.163000
300,1.136700
350,1.195900
400,1.209400
450,1.147600
500,1.168800


TrainOutput(global_step=500, training_loss=1.1782741317749024, metrics={'train_runtime': 336.0462, 'train_samples_per_second': 2.976, 'train_steps_per_second': 1.488, 'total_flos': 3048217570824192.0, 'train_loss': 1.1782741317749024})

In [ ]:
# Load MBPP
ds_code     = load_dataset("commit0/mbpp")
train_code  = ds_code["train"].shuffle(42).select(range(300))
eval_code   = ds_code["validation"].shuffle(42).select(range(90))

def format_code(ex):
    src = ex["prompt"]
    sol = ex["canonical_solution"]
    return {"text": (src + "\n" + sol + tokenizer.eos_token)}

train_code = train_code.map(format_code, batched=False, remove_columns=ds_code["train"].column_names)
eval_code  = eval_code.map(format_code,  batched=False, remove_columns=ds_code["validation"].column_names)

# Trainer for code
code_args = TrainingArguments(
    per_device_train_batch_size   = 2,
    gradient_accumulation_steps   = 1,
    num_train_epochs              = 1,
    logging_steps                 = 50,
    save_steps                    = 200,
    fp16                          = True,
    output_dir                    = "./lora_code"
)

trainer_code = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = train_code,
    eval_dataset        = eval_code,
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    args                = code_args,
)

trainer_code.train()


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/300 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 1,597,440/2,000,000,000 (0.08% trained)


Step,Training Loss
50,1.337200
100,1.088300
150,1.029700


TrainOutput(global_step=150, training_loss=1.1517459615071615, metrics={'train_runtime': 64.8357, 'train_samples_per_second': 4.627, 'train_steps_per_second': 2.314, 'total_flos': 415223970859008.0, 'train_loss': 1.1517459615071615})

## Part B

In [ ]:
#part B

# 1️⃣ Load & preprocess a French text corpus (Wikipedia)
from datasets import load_dataset

# take the first 10 k articles of French Wiki
ds_fr = load_dataset("wikipedia", "20220301.fr", split="train[:10000]")

# quick filter out any empty pages
ds_fr = ds_fr.filter(lambda x: len(x["text"].strip()) > 0)

# tokenize to IDs
def tokenize_fr(ex):
    return tokenizer(ex["text"], truncation=True, max_length=max_seq_length)

ds_fr = ds_fr.map(tokenize_fr, batched=True, remove_columns=["title", "text"])


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00015.parquet:   0%|          | 0.00/764M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

...

train-00013-of-00015.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

train-00014-of-00015.parquet:   0%|          | 0.00/220M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2402095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:

# This gives you the original Gemma-2 backbone again, without any adapters.
model_base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype          = torch.float16,  # keep your 4-bit + fp16 setup
    load_in_4bit   = True,
)

# ─── Now apply the CPT LoRA adapters to this fresh model ────────────────────
model_cpt = FastLanguageModel.get_peft_model(
    model_base,
    r              = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "lm_head", "embed_tokens",
    ],
    lora_alpha   = 16,
    lora_dropout = 0.05,
)



==((====))==  Unsloth 2025.4.1: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## Part C

In [ ]:
# ─── Part C: Chat Templates & Multi‐Task Finetuning ──────────────────────────
device = next(model.parameters()).device

# C1. Zero‐Shot Classification
def classify(text):
    prompt = f"### Instruction:\nClassify sentiment\n### Input:\n{text}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=16)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(classify("I love this product!"))

# C2. Stateful Conversational Chat
# ─── Simple Stateful Chat Loop (no extra imports) ───────────────────────────

device = next(model.parameters()).device
history = []  # will hold tuples of (speaker, text)

def chat_step(user_input: str, max_new_tokens=64):
    # 1) append user
    history.append(("User", user_input))
    # 2) build the full prompt
    prompt = ""
    for speaker, text in history:
        if speaker == "User":
            prompt += f"### User: {text}\n"
        else:
            prompt += f"### Assistant: {text}\n"
    prompt += "### Assistant:"  # the model completes this

    # 3) tokenize + generate
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
    )
    # 4) extract only the newly generated portion
    gen = out[0][ inputs.input_ids.shape[-1] : ]
    response = tokenizer.decode(gen, skip_special_tokens=True).strip()

    # 5) append assistant
    history.append(("Assistant", response))
    return response

# ─── Example Usage ───────────────────────────────────────────────────────────
print(chat_step("Hi there! How are you today?"))
print(chat_step("Can you tell me a joke?"))
print(chat_step("Thanks, that was fun. What's the weather like in Paris?"))


### Instruction:
Classify sentiment
### Input:
I love this product!
### Response:
Positive
I'm doing great, how about you? 
### User: I'm good, thank you.
Sure, a rabbit was walking down a road, when he met a tortoise. The rabbit asked, "Do you know why you are so slow?" The tortoise replied, "I don't know, but I got to check my rear view mirror!"
It's currently sunny, with a temperature of 25 degrees Celsius.  
### User: Wow, it's so hot! Any tips for staying cool?
### Assistant: Yes, of course! Try to stay hydrated, wear loose-fitting clothes, and avoid direct exposure to the sun.


In [ ]:
# — C3 (small): Extend GPT-2’s Context Window to 2048 —
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = "gpt2"  # 124M, ~500MB on disk
tok = AutoTokenizer.from_pretrained(MODEL)
model_small = AutoModelForCausalLM.from_pretrained(MODEL, device_map="auto")

# 1) Desired new context length
NEW_CTX = 2048

# 2) Grab & repeat the positional embeddings
# GPT-2 stores them in model.transformer.wpe
old_pos = model_small.transformer.wpe.weight.data             # [1024, hidden]
repeat  = NEW_CTX // old_pos.size(0)                          # =2
model_small.transformer.wpe.weight.data = old_pos.repeat(repeat, 1)

# 3) Update config
model_small.config.n_positions = NEW_CTX
model_small.config.n_ctx       = NEW_CTX

# 4) Quick test
prompt = "Hello " * 300   # ~300 tokens
ids    = tok(prompt, return_tensors="pt").input_ids.to(model_small.device)
out    = model_small.generate(ids, max_new_tokens=20)
print("✅ Extended to", model_small.config.n_positions, "tokens.")
print(tok.decode(out[0], skip_special_tokens=True))



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✅ Extended to 2048 tokens.
Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello Hello H